<a href="https://colab.research.google.com/github/ronak0001/Data-Extraction-and-Handwritten-Data-Recognition/blob/master/HandwrittenRecognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install kaggle

In [0]:
import json

token = {"username":"your_username","key":"your_key"}

with open("/root/.kaggle/kaggle.json", "w") as file:
  json.dump(token, file)

In [0]:
!chmod 600 /root/.kaggle/kaggle.json

In [0]:
!kaggle datasets download -d crawford/emnist

In [0]:
!unzip emnist.zip

In [0]:

import numpy as np
from numpy import genfromtxt
import matplotlib.pyplot as plt


In [0]:
digits_train_file = 'emnist-digits-train.csv'
digits_test_file = 'emnist-digits-test.csv'
letters_train_file = 'emnist-letters-train.csv'
letters_test_file = 'emnist-letters-test.csv'

# Read Dataset
letters_test = genfromtxt(letters_test_file, delimiter=',')
letters_train = genfromtxt(letters_train_file, delimiter=',')
digits_test = genfromtxt(digits_test_file, delimiter=',')
digits_train = genfromtxt(digits_train_file, delimiter=',')

# Save Data
np.save('letters_test.npy', letters_test)
np.save('letters_train.npy', letters_train)
np.save('digits_test.npy', digits_test)
np.save('digits_train.npy', digits_train)

# Load Data
digits_test = np.load('digits_test.npy')
plt.figure()
for i in range(0,20):
    print(digits_test[i, 0])
    plt.subplot(4,5,(i+1))
    plt.imshow(digits_test[i, 1:786].reshape(28,28))
    plt.grid(False)

plt.show()

In [0]:
import numpy as np
import matplotlib.pyplot as plt
from keras import Sequential
from keras.activations import relu, softmax
from keras.layers import Flatten, Dense, Dropout
from keras.datasets import mnist


letters_train = np.load('letters_train.npy')
letters_test = np.load('letters_test.npy
""""
(train_data, train_labels), (test_data, test_labels) = mnist.load_data()
len = letters_train.shape[0]

for i in range(0, len):
    train_data = np.append(train_data, [letters_train[i, 1:786].reshape(28, 28)], axis=0)
    train_labels = np.append(train_labels, [(letters_train[i, 0]+9)], axis=0)
print(train_data.shape)
print(train_labels.shape)

for i in range(0, letters_test.shape[0]):
    test_data = np.append(test_data, [letters_test[i, 1:786].reshape(28, 28)], axis=0)
    test_labels = np.append(test_labels, [(letters_test[i, 0]+9)], axis=0)
print(test_data.shape)
print(test_labels.shape)
"""

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
import cv2

#keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils
import sklearn.metrics as metrics

train = pd.read_csv("emnist-balanced-train.csv",delimiter = ',')
test = pd.read_csv("emnist-balanced-test.csv", delimiter = ',')
mapp = pd.read_csv("emnist-balanced-mapping.txt", delimiter = ' ', \
                   index_col=0, header=None, squeeze=True)

print("Train: %s, Test: %s, Map: %s" %(train.shape, test.shape, mapp.shape))

In [0]:
# Constants
HEIGHT = 28
WIDTH = 28

# Split x and y
train_x = train.iloc[:,1:]
train_y = train.iloc[:,0]
del train

test_x = test.iloc[:,1:]
test_y = test.iloc[:,0]
del test

In [0]:
print(train_x.shape,train_y.shape,test_x.shape,test_y.shape)

In [0]:
def rotate(image):
    image = image.reshape([HEIGHT, WIDTH])
    image = np.fliplr(image)
    image = np.rot90(image)
    return image

In [0]:
# Flip and rotate image
train_x = np.asarray(train_x)
train_x = np.apply_along_axis(rotate, 1, train_x)
print ("train_x:",train_x.shape)

test_x = np.asarray(test_x)
test_x = np.apply_along_axis(rotate, 1, test_x)
print ("test_x:",test_x.shape)

In [0]:
# Normalise
train_x = train_x.astype('float32')
train_x /= 255
test_x = test_x.astype('float32')
test_x /= 255

In [0]:
 # plot image
#  plt.imshow(train_x[1], cmap=plt.get_cmap('gray'))
# plot image
for i in range(100, 109):
    plt.subplot(330 + (i+1))
    plt.imshow(train_x[i], cmap=plt.get_cmap('gray'))
    plt.title(chr(mapp[train_y[i]]))

In [0]:
# One hot encoding
train_y = np_utils.to_categorical(train_y, num_classes)
test_y = np_utils.to_categorical(test_y, num_classes)
print("train_y: ", train_y.shape)
print("test_y: ", test_y.shape)

In [0]:
# Reshape image for CNN
train_x = train_x.reshape(-1, HEIGHT, WIDTH, 1)
test_x = test_x.reshape(-1, HEIGHT, WIDTH, 1)


In [0]:
# partition to train and val
train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size= 0.10, random_state=7)

In [0]:
# Building model
# ((Si - Fi + 2P)/S) + 1
model = Sequential()

model.add(Conv2D(filters=128, kernel_size=(5,5), padding = 'same', activation='relu',\
                 input_shape=(HEIGHT, WIDTH,1)))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(Conv2D(filters=64, kernel_size=(3,3) , padding = 'same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(units=128, activation='relu'))
model.add(Dropout(.5))
model.add(Dense(units=num_classes, activation='softmax'))

model.summary()

In [0]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [0]:
history = model.fit(train_x, train_y, epochs=10, batch_size=512, verbose=1, \
                    validation_data=(val_x, val_y))

In [0]:
model.save('terminators.hdf5')

In [0]:
def plot_accuracy(history,title):
    plt.title(title)
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train_accuracy', 'validation_accuracy'], loc='best')
    plt.show()
def plot_loss(history,title):
    plt.title(title)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train_loss', 'validation_loss'], loc='best')
    plt.show()
    
    
def predict_class(model, images, show = True):
  for img in images:
    img = image.load_img(img, target_size=(28, 28))
    img = image.img_to_array(img)                    
    img = np.expand_dims(img, axis=0)         
    img /= 255.                                      

    pred = model.predict(img)
    index = np.argmax(pred)
    food_list.sort()
    pred_value = food_list[index]
    if show:
        plt.imshow(img[0])                           
        plt.axis('off')
        plt.title(pred_value)
        plt.show()

In [0]:
plot_accuracy(history,'Terminators')
plot_loss(history,'Terminators')


In [0]:
%%time
# Loading the best saved model to make predictions
from tensorflow.keras.models import load_model
#from tensorflow.keras import models
#K.clear_session()
model_best = load_model('terminators.hdf5',compile = False)

In [0]:
print(test_x.shape)

In [0]:
y_pred = model.predict(test_x)

y_pred = (y_pred > 0.5)


In [0]:
print(test_y.shape)

In [0]:
print(y_pred[1])
print(test_y[1])

print(y_pred[5])
print(test_y[5])

In [0]:
img = test_x[1].reshape(1, 28, 28, 1)
img = img.reshape(1, 28, 28, 1)
	# prepare pixel data
img = img.astype('float32')
img = img / 255.0

print(img.shape)
plt.imshow(img)

In [0]:
!wget -O seven.jpg https://3qeqpr26caki16dnhd19sv6by6v-wpengine.netdna-ssl.com/wp-content/uploads/2019/02/sample_image.png
  


In [0]:
# make a prediction for a new image.
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.models import load_model

# load and prepare the image
img = cv2.imread('seven.jpg')
#img = np.asarray(img)
#img.astype('float32')
plt.imshow(img)
plt.show()
def load_image(filename):
	# load the image
	img = load_img(filename, grayscale=True, target_size=(28, 28))
	# convert to array
	img = img_to_array(img)
	# reshape into a single sample with 1 channel
	img = img.reshape(1, 28, 28, 1)
	# prepare pixel data
	img = img.astype('float32')
	img = img / 255.0
	return img

# load an image and predict the class
def run_example():
	# load the image
	img = load_image('seven.jpg')
	# load model
	model = load_model('terminators.hdf5')
	# predict the class
	digit = model.predict_classes(img)
	print(digit[0])
  
# entry point, run the example
run_example()